In [1]:
import pandas
!pip install folium
import folium
import math
from folium.plugins import HeatMap

data = pandas.read_csv("AB_NYC_2019.csv")
data = data.sort_values(["price"])


# visualizations 
1. Heatmap based on prices
2. Heatmap based on availability 
3. Room_type vs Price 
4. Relationship bewteen rating and price
5. Heatmap based on rating
6. Room_type vs availability



# Machine Learning:
1. Based on other info -> Suggested Price

In [2]:
data

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
25796,20639914,Cozy yet spacious private brownstone bedroom,86327101,Adeyemi,Brooklyn,Bedford-Stuyvesant,40.68258,-73.91284,Private room,0,1,95,2019-06-23,4.35,6,222
25634,20523843,"MARTIAL LOFT 3: REDEMPTION (upstairs, 2nd room)",15787004,Martial Loft,Brooklyn,Bushwick,40.69467,-73.92433,Private room,0,2,16,2019-05-18,0.71,5,0
25433,20333471,★Hostel Style Room | Ideal Traveling Buddies★,131697576,Anisha,Bronx,East Morrisania,40.83296,-73.88668,Private room,0,2,55,2019-06-24,2.56,4,127
25753,20608117,"Sunny, Quiet Room in Greenpoint",1641537,Lauren,Brooklyn,Greenpoint,40.72462,-73.94072,Private room,0,2,12,2017-10-27,0.53,2,0
23161,18750597,"Huge Brooklyn Brownstone Living, Close to it all.",8993084,Kimberly,Brooklyn,Bedford-Stuyvesant,40.69023,-73.95428,Private room,0,4,1,2018-01-06,0.05,4,28
25795,20639792,Contemporary bedroom in brownstone with nice view,86327101,Adeyemi,Brooklyn,Bedford-Stuyvesant,40.68279,-73.91170,Private room,0,1,95,2019-06-21,4.37,6,232
26841,21291569,Coliving in Brooklyn! Modern design / Shared room,101970559,Sergii,Brooklyn,Bushwick,40.69211,-73.90670,Shared room,0,30,2,2019-06-22,0.11,6,333
25778,20624541,Modern apartment in the heart of Williamsburg,10132166,Aymeric,Brooklyn,Williamsburg,40.70838,-73.94645,Entire home/apt,0,5,3,2018-01-02,0.15,1,73
25794,20639628,Spacious comfortable master bedroom with nice ...,86327101,Adeyemi,Brooklyn,Bedford-Stuyvesant,40.68173,-73.91342,Private room,0,1,93,2019-06-15,4.28,6,176
26866,21304320,Best Coliving space ever! Shared room.,101970559,Sergii,Brooklyn,Bushwick,40.69166,-73.90928,Shared room,0,30,5,2019-05-24,0.26,6,139


Explaining the Data

The data presents details of each Airbnb room in New York City. One row represents a Airbnb room and the columns represent the attributes of the room. The attributes/columns are Airbnb id, name of the Airbnb, host id, neighbourhood group that the room presides in, specific neighborhood within the neighborhood grouo, location of the Airbnb using latitude and longitude, the room type, price, minimum nights the Airbnb room allows guests to stay, the total number of reviews on the room, the average reviews per month, how many listings of Airbnbs does the host have, and the availibilty of the room throughout the year.

Preprocessing Data

We do not need all the columns for our data analysis so we remove columns not used

In [6]:
#Remove id, name, host_id, neighborhood,minimum nights, last review, reviews per month, calc host listings,
data = data.drop(columns=['id', 'name', 'host_id', 'host_name', 'neighbourhood', 'minimum_nights', 'last_review', 'reviews_per_month', 'calculated_host_listings_count'])

Some Airbnb prices are listed at 0. The price can be 0 or the Airbnb lister did not put a price down. We exclude prices with 0 in our data

In [8]:
data

,neighbourhood_group,latitude,longitude,room_type,price,number_of_reviews,availability_365
25796,Brooklyn,40.68258,-73.91284,Private room,0,95,222
25634,Brooklyn,40.69467,-73.92433,Private room,0,16,0
25433,Bronx,40.83296,-73.88668,Private room,0,55,127
25753,Brooklyn,40.72462,-73.94072,Private room,0,12,0
23161,Brooklyn,40.69023,-73.95428,Private room,0,1,28
25795,Brooklyn,40.68279,-73.91170,Private room,0,95,232
26841,Brooklyn,40.69211,-73.90670,Shared room,0,2,333
25778,Brooklyn,40.70838,-73.94645,Entire home/apt,0,3,73
25794,Brooklyn,40.68173,-73.91342,Private room,0,93,176
26866,Brooklyn,40.69166,-73.90928,Shared room,0,5,139


In [9]:
#Exclude rows that have prices as 0
data_with_acc_price = data[data['price'] > 0]

In [ ]:
for index, row in data_with_acc_price.iterrows():
    if(pandas.isna(row['availability_365'])):
        print(row['availability_365'])

In [11]:
#Latitude and Longitude for New York City
#48884 total values so we need to sample of 1000
data_price_sample = data_with_acc_price.sample(n=1000)
m = folium.Map(location=[40.7128, -74.0060], zoom_start=14)
heat_data = []
for index, row in data_price_sample.iterrows():
    loc_price = []
    lat = row['latitude']
    long = row['longitude']
    price = row['price']
    loc_price.append(lat)
    loc_price.append(long)
    loc_price.append(price)
    heat_data.append(loc_price)


In [5]:
HeatMap(heat_data, max_val=10000).add_to(m)
m

The heatmap represents the prices of Airbnbs in New York City. Areas with red represent expensive Airbnbs compared to green and blue areas. From the heatmap it shows that Manhattan and Brooklyn have the most expensive Airbnbs.

In [6]:
#Heatmap based on availability
#Latitude and Longitude for New York City
#48884 total values so we need to sample of 1000
data_price_sample = data_price.sample(n=1000)
m = folium.Map(location=[40.7128, -74.0060], zoom_start=14)
heat_data = []
for index, row in data_price_sample.iterrows():
    loc_availability = []
    lat = row['latitude']
    long = row['longitude']
    availability_365 = row['availability_365']
    loc_availability.append(lat)
    loc_availability.append(long)
    loc_availability.append(-availability_365)
    heat_data.append(loc_availability)



In [7]:
HeatMap(heat_data, max_val=0).add_to(m)
m

In [8]:
This heatmap represents the availabilities of Airbnbs in New York City. The max value of availabilities is 0 which means that the Airbnb is unavailable 365 days of the year. We correlate the unavailability of a Airbnb with its popularity. The red areas in the map represents areas in New York City with Airbnbs that are 

SyntaxError: invalid syntax (<ipython-input-8-b72584a43075>, line 1)